# Weather Image Classification Part Two

# Imports

In [18]:
import os

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import ImageFile, Image


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

# Functions

In [30]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

def load_process_images(img_path, target_size=(224,224)):
  
  try:
    with Image.open(img_path) as image:
      image = image.resize(target_size)
      image = image.convert("RGB")
      
      img_array = np.array(image)/255.0
      img_array = img_array.astype(np.float32)
      
      return img_array
  except IOError as e:
    print(f"Error loading image: {e}")
    return None

def grab_images(PATH):
  images, labels = [], []
  for root, dir, files in os.walk(PATH):
    for file in files:
      if file.lower().endswith((".png", ".jpeg", ".jpg")):
        file_path = os.path.join(root, file)
        img = load_process_images(file_path)
        if img is not None:
          images.append(img)
          label = root.split(os.sep)[-1]
          labels.append(label)
  
  images, labels = np.array(images), np.array(labels)
  
  return images, labels

def create_model(n_classes):
  
  model = Sequential([
    Conv2D(32, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(n_classes, activation="softmax")
  ])
  
  model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
  )
  model.summary()
  
  return model

# Create Path

In [31]:
PATH = "/Users/maukanmir/Downloads/dataset/"

# Grab Images

In [32]:
X ,y = grab_images(PATH)

In [33]:
label_encoder = LabelEncoder()

y_int = label_encoder.fit_transform(y)

In [34]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_int, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

In [35]:
print(f'Training set size: {X_train.shape[0]}')
print(f'Validation set size: {X_val.shape[0]}')
print(f'Test set size: {X_test.shape[0]}')

Training set size: 4116
Validation set size: 1373
Test set size: 1373


# Data Augmentation

In [ ]:
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode="nearest"
)

validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(
  X_train, y_train,
  batch_size=BATCH_SIZE
)

validation_generator = validation_datagen.flow(
  X_val, y_val,
  batch_size=BATCH_SIZE
)

test_generator = test_datagen.flow(
  X_test, y_test,
  batch_size=BATCH_SIZE
)